## Instalação das Bibliotecas

In [ ]:
!pip install pandas requests plotly

## Importação das Bibliotecas

In [ ]:
import pandas as pd
import requests
import plotly.express as px
from datetime import datetime

## Função para coletar dados do Banco Central do Brasil

In [ ]:

def coletar_bcb(codigo, data_inicial, data_final=None):
    """Coleta dados do BCB via API."""
    if data_final is None:
        data_final = datetime.today().strftime('%d/%m/%Y')
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados'
    params = {'formato': 'json', 'dataInicial': data_inicial, 'dataFinal': data_final}
    try:
        resp = requests.get(url, params=params, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        resp.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f'Erro ao acessar API: {e}')
    df = pd.DataFrame(resp.json())
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
    df = df.dropna()
    return df


## Coleta dos Indicadores

In [ ]:

selic = coletar_bcb(4189, '01/01/2015')
ipca = coletar_bcb(433, '01/01/2015')
fx = coletar_bcb(1, '01/01/2015')
pib = coletar_bcb(4380, '01/01/2015')


## Visualizações Interativas

In [ ]:

fig_selic = px.line(selic, x='data', y='valor', title='Taxa Selic Meta')
fig_selic.update_layout(xaxis_title='Data', yaxis_title='% ao ano')
fig_selic.show()

fig_ipca = px.bar(ipca, x='data', y='valor', title='IPCA Mensal')
fig_ipca.update_layout(xaxis_title='Data', yaxis_title='% ao mês')
fig_ipca.show()

fig_fx = px.line(fx, x='data', y='valor', title='Câmbio USD/BRL')
fig_fx.update_layout(xaxis_title='Data', yaxis_title='Reais por Dólar')
fig_fx.show()

fig_pib = px.bar(pib, x='data', y='valor', title='PIB Trimestral (R$ bilhões)')
fig_pib.update_layout(xaxis_title='Data', yaxis_title='R$ bilhões')
fig_pib.show()


## Análise Automática Simples

In [ ]:

print('Última Selic registrada:', selic.iloc[-1]['valor'])
print('Último IPCA mensal:', ipca.iloc[-1]['valor'])
print('Último câmbio USD/BRL:', fx.iloc[-1]['valor'])
print('Último PIB trimestral (R$ bi):', pib.iloc[-1]['valor'])
